In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

2023-04-25 09:20:28.480361: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-25 09:20:28.524192: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-25 09:20:28.524722: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 09:20:29.138401: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Specify each path

In [2]:
dataset = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.hdf5'
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

# Set number of classes

In [3]:
NUM_CLASSES = 9

# Dataset reading

In [4]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [5]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# Model building

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [8]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout (Dropout)           (None, 42)                0         
                                                                 
 dense (Dense)               (None, 20)                860       
                                                                 
 dropout_1 (Dropout)         (None, 20)                0         
                                                                 
 dense_1 (Dense)             (None, 10)                210       
                                                                 
 dense_2 (Dense)             (None, 9)                 99        
                                                                 
Total params: 1,169
Trainable params: 1,169
Non-trainable params: 0
_________________________________________________________________


In [9]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [10]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [11]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
 1/20 [>.............................] - ETA: 9s - loss: 2.2419 - accuracy: 0.1094
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 1s 11ms/step - loss: 2.1677 - accuracy: 0.1582 - val_loss: 2.0873 - val_accuracy: 0.1444
Epoch 2/1000
 1/20 [>.............................] - ETA: 0s - loss: 2.1513 - accuracy: 0.1797
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 2.0793 - accuracy: 0.2165 - val_loss: 1.9951 - val_accuracy: 0.2816
Epoch 3/1000
 1/20 [>.............................] - ETA: 0s - loss: 2.0485 - accuracy: 0.2188
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 1.9862 - accuracy: 0.2493 - val_loss: 1.8790 - val_accuracy: 0.4296
Epoch 4/1000
 1/20 [>.............................] - ETA: 0s - loss: 1.9576 - accuracy: 0.2266


 1/20 [>.............................] - ETA: 0s - loss: 0.8444 - accuracy: 0.6641
Epoch 28: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.9274 - accuracy: 0.6374 - val_loss: 0.6883 - val_accuracy: 0.8434
Epoch 29/1000
 1/20 [>.............................] - ETA: 0s - loss: 1.0130 - accuracy: 0.5781
Epoch 29: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.9186 - accuracy: 0.6261 - val_loss: 0.6706 - val_accuracy: 0.8495
Epoch 30/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.9408 - accuracy: 0.6719
Epoch 30: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.9114 - accuracy: 0.6398 - val_loss: 0.6511 - val_accuracy: 0.8677
Epoch 31/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.8379 - accuracy: 0.6953
Epoch 31

 1/20 [>.............................] - ETA: 0s - loss: 0.5909 - accuracy: 0.7578
Epoch 55: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.7240 - accuracy: 0.7236 - val_loss: 0.4104 - val_accuracy: 0.9575
Epoch 56/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.5829 - accuracy: 0.8047
Epoch 56: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.7122 - accuracy: 0.7272 - val_loss: 0.4100 - val_accuracy: 0.9600
Epoch 57/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.7116 - accuracy: 0.7188
Epoch 57: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.7040 - accuracy: 0.7228 - val_loss: 0.4101 - val_accuracy: 0.9600
Epoch 58/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.7806 - accuracy: 0.7344
Epoch 58

 1/20 [>.............................] - ETA: 0s - loss: 0.5130 - accuracy: 0.8359
Epoch 82: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.6118 - accuracy: 0.7661 - val_loss: 0.3280 - val_accuracy: 0.9660
Epoch 83/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.6490 - accuracy: 0.7109
Epoch 83: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.6527 - accuracy: 0.7503 - val_loss: 0.3276 - val_accuracy: 0.9660
Epoch 84/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.7868 - accuracy: 0.7188
Epoch 84: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.6470 - accuracy: 0.7584 - val_loss: 0.3330 - val_accuracy: 0.9684
Epoch 85/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.5696 - accuracy: 0.7812
Epoch 85

 1/20 [>.............................] - ETA: 0s - loss: 0.6821 - accuracy: 0.7188
Epoch 109: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.6049 - accuracy: 0.7746 - val_loss: 0.2932 - val_accuracy: 0.9697
Epoch 110/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.5376 - accuracy: 0.8359
Epoch 110: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.5872 - accuracy: 0.7782 - val_loss: 0.2845 - val_accuracy: 0.9733
Epoch 111/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.6772 - accuracy: 0.7266
Epoch 111: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.5914 - accuracy: 0.7798 - val_loss: 0.2805 - val_accuracy: 0.9733
Epoch 112/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.6699 - accuracy: 0.6875
Ep

...

Epoch 217/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.5162 - accuracy: 0.8047
Epoch 217: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.5279 - accuracy: 0.8122 - val_loss: 0.2048 - val_accuracy: 0.9757
Epoch 218/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.5084 - accuracy: 0.8438
Epoch 218: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.5067 - accuracy: 0.8183 - val_loss: 0.2030 - val_accuracy: 0.9733
Epoch 219/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.4998 - accuracy: 0.8125
Epoch 219: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 4ms/step - loss: 0.5238 - accuracy: 0.8070 - val_loss: 0.2031 - val_accuracy: 0.9745
Epoch 220/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.4530 - accu

Epoch 244/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.2659 - accuracy: 0.8984
Epoch 244: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.5066 - accuracy: 0.8155 - val_loss: 0.1851 - val_accuracy: 0.9794
Epoch 245/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.5925 - accuracy: 0.8047
Epoch 245: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 4ms/step - loss: 0.4830 - accuracy: 0.8227 - val_loss: 0.1849 - val_accuracy: 0.9769
Epoch 246/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.4465 - accuracy: 0.8750
Epoch 246: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.4863 - accuracy: 0.8215 - val_loss: 0.1832 - val_accuracy: 0.9794
Epoch 247/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.4635 - accu

Epoch 271/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.6049 - accuracy: 0.7891
Epoch 271: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.4955 - accuracy: 0.8244 - val_loss: 0.1843 - val_accuracy: 0.9745
Epoch 272/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.5587 - accuracy: 0.7891
Epoch 272: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.4919 - accuracy: 0.8268 - val_loss: 0.1832 - val_accuracy: 0.9769
Epoch 273/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.4908 - accuracy: 0.8047
Epoch 273: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.4859 - accuracy: 0.8195 - val_loss: 0.1781 - val_accuracy: 0.9782
Epoch 274/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.5609 - accu

Epoch 298/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.5654 - accuracy: 0.8281
Epoch 298: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.4582 - accuracy: 0.8442 - val_loss: 0.1736 - val_accuracy: 0.9782
Epoch 299/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.5658 - accuracy: 0.8047
Epoch 299: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.4764 - accuracy: 0.8361 - val_loss: 0.1787 - val_accuracy: 0.9745
Epoch 300/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.4399 - accuracy: 0.8516
Epoch 300: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.4790 - accuracy: 0.8292 - val_loss: 0.1758 - val_accuracy: 0.9806
Epoch 301/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.4476 - accu

Epoch 325/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.5373 - accuracy: 0.8281
Epoch 325: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.4708 - accuracy: 0.8329 - val_loss: 0.1771 - val_accuracy: 0.9769
Epoch 326/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.5124 - accuracy: 0.8594
Epoch 326: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.4773 - accuracy: 0.8296 - val_loss: 0.1882 - val_accuracy: 0.9672
Epoch 327/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.4965 - accuracy: 0.7891
Epoch 327: saving model to model/keypoint_classifier/keypoint_classifier.hdf5
20/20 [==============================] - 0s 3ms/step - loss: 0.4511 - accuracy: 0.8308 - val_loss: 0.1727 - val_accuracy: 0.9745
Epoch 328/1000
 1/20 [>.............................] - ETA: 0s - loss: 0.4382 - accu

In [12]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

7/7 [==============================] - 0s 1ms/step - loss: 0.1676 - accuracy: 0.9782


In [13]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [14]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 [==============================] - 0s 59ms/step
[1.5679752e-05 5.3122329e-07 6.0010434e-06 9.8576290e-13 1.5909123e-07
 1.8900622e-07 1.1384163e-01 2.3514062e-02 8.6262172e-01]
8


# Saving Model

In [15]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

In [16]:
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

2023-04-25 09:21:12.297855: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_1' with dtype float and shape [?,42]
	 [[{{node input_1}}]]
2023-04-25 09:21:12.348123: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,42]
	 [[{{node inputs}}]]
2023-04-25 09:21:12.359444: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'input_1' with dtype float and shape [?,42]
	 [[{{node input_1}}]]
2023-04-25 09:2

INFO:tensorflow:Assets written to: /tmp/tmps8d6uxkm/assets


INFO:tensorflow:Assets written to: /tmp/tmps8d6uxkm/assets
2023-04-25 09:21:13.237195: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-04-25 09:21:13.237232: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2023-04-25 09:21:13.237533: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmps8d6uxkm
2023-04-25 09:21:13.238596: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-04-25 09:21:13.238612: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmps8d6uxkm
2023-04-25 09:21:13.242067: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-04-25 09:21:13.243066: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2023-04-25 09:21:13.275717: I tensorflow/cc/saved_model/loader.cc:215] Running initializatio

6764

# Inference test

In [17]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [18]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [19]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [20]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

CPU times: user 0 ns, sys: 126 µs, total: 126 µs
Wall time: 85.8 µs


In [21]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[1.5679751e-05 5.3122432e-07 6.0010489e-06 9.8576290e-13 1.5909137e-07
 1.8900640e-07 1.1384166e-01 2.3514133e-02 8.6262172e-01]
8
